# Frequentist Model Selection with AIC/BIC

This notebook demonstrates systematic model comparison using **frequentist information criteria** (AIC, BIC) and the `ModelComparisonPipeline` API. This complements the Bayesian approach (WAIC/LOO) shown in `bayesian/04-model-comparison.ipynb`.

**Learning Objectives:**
- Use `ModelComparisonPipeline` for automated model comparison
- Understand AIC (Akaike Information Criterion) and BIC (Bayesian Information Criterion)
- Calculate AIC weights and evidence ratios for model selection
- Interpret complexity vs performance trade-offs
- Compare frequentist (AIC/BIC) vs Bayesian (WAIC/LOO) model selection

**Prerequisites:** Basic understanding of model fitting (complete basic/ notebooks first)

**Estimated Time:** 40-45 minutes

**Companion Notebook:** Use `bayesian/04-model-comparison.ipynb` for Bayesian model selection with WAIC/LOO

**Key Concepts:**
- **AIC** (Akaike Information Criterion): Penalizes complexity based on number of parameters
- **BIC** (Bayesian Information Criterion): Stronger complexity penalty than AIC
- **ΔAIC < 2**: Models essentially equivalent
- **2 < ΔAIC < 4**: Moderate evidence for best model
- **4 < ΔAIC < 7**: Strong evidence for best model
- **ΔAIC > 10**: Very strong evidence for best model

## Setup and Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from rheo.pipeline import ModelComparisonPipeline
from rheo.core.data import RheoData

np.random.seed(42)

## Generate Synthetic Polymer Oscillation Data

We'll create frequency sweep data (G' and G") that resembles a viscoelastic polymer.

In [ ]:
# Generate frequency vector
frequency = np.logspace(-2, 3, 50)  # 0.01 to 1000 rad/s

# True underlying model: Fractional Zener (SLS variant)
# Parameters
Ge_true = 1e5      # Equilibrium modulus (Pa)
Gm_true = 5e5      # Maxwell arm modulus (Pa)
alpha_true = 0.6   # Fractional order
tau_true = 1.0     # Relaxation time (s)

# Calculate G' and G" (simplified fractional Zener)
omega = frequency
omega_tau = omega * tau_true

# Storage modulus G'
G_storage_true = Ge_true + Gm_true * (omega_tau)**alpha_true / (1 + (omega_tau)**(2*alpha_true))

# Loss modulus G"
G_loss_true = Gm_true * (omega_tau)**alpha_true / (1 + (omega_tau)**(2*alpha_true)) * np.sin(alpha_true * np.pi/2)

# Add realistic noise (5% for G', 7% for G")
noise_storage = 0.05
noise_loss = 0.07

G_storage_noisy = G_storage_true * (1 + noise_storage * np.random.randn(len(G_storage_true)))
G_loss_noisy = G_loss_true * (1 + noise_loss * np.random.randn(len(G_loss_true)))

# For model fitting, we'll use |G*| (complex modulus magnitude)
G_star_true = np.sqrt(G_storage_true**2 + G_loss_true**2)
G_star_noisy = np.sqrt(G_storage_noisy**2 + G_loss_noisy**2)

# Create RheoData object
data = RheoData(
    x=frequency,
    y=G_star_noisy,
    x_units='rad/s',
    y_units='Pa',
    domain='frequency',
    test_mode='oscillation'
)

print(f"Generated {len(frequency)} data points")
print(f"Frequency range: {frequency[0]:.2e} to {frequency[-1]:.2e} rad/s")
print(f"Complex modulus range: {G_star_noisy.min():.2e} to {G_star_noisy.max():.2e} Pa")

## Visualize the Data

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: G' and G"
ax1.loglog(frequency, G_storage_noisy, 'o', label="G' (storage)", markersize=6, alpha=0.7)
ax1.loglog(frequency, G_loss_noisy, 's', label='G" (loss)', markersize=6, alpha=0.7)
ax1.set_xlabel('Frequency (rad/s)', fontsize=12)
ax1.set_ylabel('Modulus (Pa)', fontsize=12)
ax1.set_title('Storage and Loss Moduli', fontsize=13)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Right: |G*| (complex modulus)
ax2.loglog(frequency, G_star_noisy, 'o', color='purple', 
           label='|G*| (data)', markersize=6, alpha=0.7)
ax2.loglog(frequency, G_star_true, '--', color='red', 
           label='True model', linewidth=2)
ax2.set_xlabel('Frequency (rad/s)', fontsize=12)
ax2.set_ylabel('|G*| (Pa)', fontsize=12)
ax2.set_title('Complex Modulus Magnitude', fontsize=13)
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Define Candidate Models

We'll compare 5 models of increasing complexity:
1. **Maxwell**: Simplest viscoelastic model
2. **Zener**: Standard Linear Solid with equilibrium modulus
3. **Fractional Maxwell (Gel)**: Power-law relaxation
4. **Fractional Maxwell (Liquid)**: Fractional Maxwell without equilibrium modulus
5. **Fractional Zener (SS)**: Most complex, with fractional elements

In [ ]:
# Define candidate models
candidate_models = [
    'maxwell',
    'zener',
    'fractional_maxwell_gel',
    'fractional_maxwell_liquid',
    'fractional_zener_ss'
]

print(f"Comparing {len(candidate_models)} models:")
for i, model in enumerate(candidate_models, 1):
    print(f"  {i}. {model}")

## Run Model Comparison Pipeline

The `ModelComparisonPipeline` automatically fits all models and computes comparison metrics.

In [ ]:
# Create and run comparison pipeline
comparison_pipeline = ModelComparisonPipeline(candidate_models)
results = comparison_pipeline.run(data)

print("\nModel comparison complete!")
print(f"Fitted {len(results)} models successfully")

## Display Comparison Table

We'll create a comprehensive comparison table with:
- **RMSE**: Root Mean Square Error (lower is better)
- **R²**: Coefficient of determination (higher is better, max 1.0)
- **AIC**: Akaike Information Criterion (lower is better)
- **BIC**: Bayesian Information Criterion (lower is better)

In [ ]:
# Extract metrics for each model
comparison_data = []

for model_name in candidate_models:
    if model_name in results:
        result = results[model_name]
        comparison_data.append({
            'Model': model_name,
            'Parameters': result['n_params'],
            'RMSE': result['rmse'],
            'R²': result['r_squared'],
            'AIC': result['aic'],
            'BIC': result['bic']
        })

# Create DataFrame
df_comparison = pd.DataFrame(comparison_data)

# Sort by AIC (lower is better)
df_comparison_sorted = df_comparison.sort_values('AIC').reset_index(drop=True)

# Display formatted table
print("\n" + "="*80)
print("MODEL COMPARISON RESULTS (sorted by AIC)")
print("="*80)
print()
print(df_comparison_sorted.to_string(index=False))
print("\n" + "="*80)
print("Lower AIC/BIC and RMSE are better; Higher R² is better (max 1.0)")
print("="*80)

## Identify Best Model

In [ ]:
# Get best model by different criteria
best_aic = comparison_pipeline.get_best_model(metric='aic')
best_bic = comparison_pipeline.get_best_model(metric='bic')
best_r2 = comparison_pipeline.get_best_model(metric='r_squared')

print("\n" + "="*60)
print("BEST MODEL BY DIFFERENT CRITERIA")
print("="*60)
print(f"\nBest by AIC:  {best_aic}")
print(f"Best by BIC:  {best_bic}")
print(f"Best by R²:   {best_r2}")

# Calculate AIC weights (relative likelihood)
aic_values = df_comparison_sorted['AIC'].values
delta_aic = aic_values - aic_values.min()
aic_weights = np.exp(-0.5 * delta_aic) / np.sum(np.exp(-0.5 * delta_aic))

print(f"\n{'Model':<30} {'AIC Weight':<15} {'Evidence Ratio'}")
print("-" * 60)
for i, (model, weight) in enumerate(zip(df_comparison_sorted['Model'], aic_weights)):
    evidence_ratio = aic_weights[0] / weight if weight > 0 else np.inf
    print(f"{model:<30} {weight:>10.4f}     {evidence_ratio:>8.1f}x")

print("\nNote: AIC weight represents the probability that the model is the best model.")
print("Evidence ratio shows how much better the best model is compared to others.")

## Visualize All Model Fits

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

colors = plt.cm.tab10(range(len(candidate_models)))

for i, model_name in enumerate(candidate_models):
    ax = axes[i]
    
    # Plot data
    ax.loglog(frequency, G_star_noisy, 'o', color='gray', 
              label='Data', markersize=5, alpha=0.5)
    
    # Plot model fit
    if model_name in results:
        predictions = results[model_name]['predictions']
        r2 = results[model_name]['r_squared']
        aic = results[model_name]['aic']
        
        ax.loglog(frequency, predictions, '-', color=colors[i], 
                 linewidth=2.5, label=f'Fit (R²={r2:.4f})')
        
        ax.set_xlabel('Frequency (rad/s)', fontsize=10)
        ax.set_ylabel('|G*| (Pa)', fontsize=10)
        ax.set_title(f'{model_name}\nAIC = {aic:.1f}', fontsize=11)
        ax.legend(fontsize=9)
        ax.grid(True, alpha=0.3)

# Remove extra subplot
fig.delaxes(axes[-1])

plt.suptitle('Model Comparison: All Fits', fontsize=14, y=1.00)
plt.tight_layout()
plt.show()

## Residual Analysis

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(16, 9))
axes = axes.flatten()

for i, model_name in enumerate(candidate_models):
    ax = axes[i]
    
    if model_name in results:
        predictions = results[model_name]['predictions']
        residuals = (G_star_noisy - predictions) / G_star_noisy * 100  # Percent error
        rmse = results[model_name]['rmse']
        
        ax.semilogx(frequency, residuals, 'o-', color=colors[i], 
                   markersize=5, alpha=0.7, linewidth=1.5)
        ax.axhline(y=0, color='k', linestyle='--', linewidth=1)
        ax.axhline(y=5, color='r', linestyle=':', linewidth=1, alpha=0.5)
        ax.axhline(y=-5, color='r', linestyle=':', linewidth=1, alpha=0.5)
        
        ax.set_xlabel('Frequency (rad/s)', fontsize=10)
        ax.set_ylabel('Relative Error (%)', fontsize=10)
        ax.set_title(f'{model_name}\nRMSE = {rmse:.2e} Pa', fontsize=11)
        ax.set_ylim([-15, 15])
        ax.grid(True, alpha=0.3)

# Remove extra subplot
fig.delaxes(axes[-1])

plt.suptitle('Residual Analysis (red lines = ±5% error)', fontsize=14, y=1.00)
plt.tight_layout()
plt.show()

## Parameter Comparison Across Models

Compare fitted parameters across models to understand physical interpretations.

In [ ]:
print("\n" + "="*80)
print("FITTED PARAMETERS FOR EACH MODEL")
print("="*80)

for model_name in candidate_models:
    if model_name in results:
        print(f"\n{model_name.upper()}:")
        print("-" * 60)
        params = results[model_name]['parameters']
        for param_name, param in params.items():
            print(f"  {param_name:<15} = {param.value:>12.3e}  {param.units if param.units else ''}")

## Complexity vs Performance Trade-off

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: R² vs number of parameters
ax1.plot(df_comparison_sorted['Parameters'], df_comparison_sorted['R²'], 
         'o-', markersize=10, linewidth=2)
for i, model in enumerate(df_comparison_sorted['Model']):
    ax1.annotate(model, 
                (df_comparison_sorted['Parameters'].iloc[i], 
                 df_comparison_sorted['R²'].iloc[i]),
                textcoords="offset points", xytext=(0,10), 
                ha='center', fontsize=8)
ax1.set_xlabel('Number of Parameters', fontsize=12)
ax1.set_ylabel('R² (Goodness of Fit)', fontsize=12)
ax1.set_title('Model Complexity vs Fit Quality', fontsize=13)
ax1.grid(True, alpha=0.3)
ax1.set_ylim([0.95, 1.001])

# Right: AIC comparison
x_pos = np.arange(len(df_comparison_sorted))
bars = ax2.bar(x_pos, df_comparison_sorted['AIC'], color=colors[:len(df_comparison_sorted)])
ax2.set_xlabel('Model', fontsize=12)
ax2.set_ylabel('AIC (lower is better)', fontsize=12)
ax2.set_title('Akaike Information Criterion', fontsize=13)
ax2.set_xticks(x_pos)
ax2.set_xticklabels(df_comparison_sorted['Model'], rotation=45, ha='right', fontsize=9)
ax2.grid(True, alpha=0.3, axis='y')

# Highlight best model
bars[0].set_edgecolor('red')
bars[0].set_linewidth(3)

plt.tight_layout()
plt.show()

## Conclusion

In this notebook, we performed systematic model comparison for rheological data:

### Key Findings:

1. **Model Selection**: Information criteria (AIC, BIC) balance fit quality against complexity
2. **Best Model**: The model with lowest AIC/BIC is statistically preferred
3. **Complexity Trade-off**: More parameters improve R² but may overfit (penalized by AIC/BIC)
4. **Residual Patterns**: Systematic residuals indicate model inadequacy

### Interpretation Guidelines:

- **ΔA IC < 2**: Models are essentially equivalent
- **2 < ΔAIC < 4**: Moderate evidence favoring best model  
- **4 < ΔAIC < 7**: Strong evidence favoring best model
- **ΔAIC > 10**: Very strong evidence favoring best model

### When to Use Each Model:

- **Maxwell**: Purely viscous liquids with single relaxation time
- **Zener**: Materials with equilibrium modulus (gels, crosslinked polymers)
- **Fractional Models**: Materials with broad relaxation spectra (power-law behavior)
- **Complex Fractional**: Multi-modal relaxation with fractional components

### Recommendations:

1. Always compare multiple models systematically
2. Use information criteria (AIC/BIC) for objective selection
3. Examine residuals for systematic patterns
4. Consider physical interpretability of parameters
5. Validate selected model with independent test data

### Next Steps:

- Apply multi-technique fitting (see `multi_technique_fitting.ipynb`)
- Perform advanced sensitivity analysis (see `advanced_workflows.ipynb`)
- Explore Bayesian model selection with uncertainty quantification (Phase 3)